---
title: "A Replication of Karlan and List (2007)"

author: "Andrew Burda"
date: 04-23-2025
callout-appearance: minimal
format:
  html:
    code-fold: false
    code-tools: false
execute:
  echo: false # this hides the blue "i" icon on .callout-notes
---



## Introduction

Dean Karlan at Yale and John List at the University of Chicago conducted a field experiment to test the effectiveness of different fundraising letters. They sent out 50,000 fundraising letters to potential donors, randomly assigning each letter to one of three treatments: a standard letter, a matching grant letter, or a challenge grant letter. They published the results of this experiment in the _American Economic Review_ in 2007. The article and supporting data are available from the [AEA website](https://www.aeaweb.org/articles?id=10.1257/aer.97.5.1774) and from Innovations for Poverty Action as part of [Harvard's Dataverse](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/27853&version=4.2).

The original experiment was conducted in partnership with a nonprofit that supported economically disadvantaged children. The researchers sent out 50,000 fundraising letters to potential donors, randomly assigning each recipient to one of three treatment groups:

- **Standard Letter (Control):** A basic appeal describing the mission of the organization and requesting support.
- **Matching Grant Treatment:** A letter stating that donations would be matched dollar-for-dollar by a lead donor, up to a specific amount.
- **Challenge Grant Treatment:** A letter explaining that a lead donor had pledged a large donation, but only if a threshold level of additional contributions was met.

The treatments were randomized to ensure internal validity, and the primary outcomes measured were:
- A binary indicator of whether a donation was made
- The dollar amount donated

By comparing outcomes across the three groups, Karlan and List aimed to uncover how different types of financial incentives (matching vs. challenge grants) influence donation behavior. The results showed that **matching grants significantly increased both the likelihood of donating and the average donation amount**, while challenge grants had a weaker and more ambiguous impact.

This experiment has since become a cornerstone in the field of behavioral economics and charitable giving, demonstrating how small shifts in message framing can meaningfully alter real-world behavior.


This project seeks to replicate their results.


## Data

### Description

This dataset contains information used in Karlan & List (2007), including treatment indicators, match ratios, suggested donation amounts, and past giving behavior. It is used to evaluate the effect of matching donations in a fundraising experiment.


In [ ]:
import pandas as pd


df = pd.read_stata("karlan_list_2007.dta")


print("=== Data Overview ===")
print(df.info())

print("\n=== First 5 Rows ===")
print(df.head())

print("\n=== Descriptive Statistics ===")
print(df.describe(include='all'))

| Variable             | Description                                                         |
|----------------------|---------------------------------------------------------------------|
| `treatment`          | Treatment                                                           |
| `control`            | Control                                                             |
| `ratio`              | Match ratio                                                         |
| `ratio2`             | 2:1 match ratio                                                     |
| `ratio3`             | 3:1 match ratio                                                     |
| `size`               | Match threshold                                                     |
| `size25`             | \$25,000 match threshold                                            |
| `size50`             | \$50,000 match threshold                                            |
| `size100`            | \$100,000 match threshold                                           |
| `sizeno`             | Unstated match threshold                                            |
| `ask`                | Suggested donation amount                                           |
| `askd1`              | Suggested donation was highest previous contribution                |
| `askd2`              | Suggested donation was 1.25 x highest previous contribution         |
| `askd3`              | Suggested donation was 1.50 x highest previous contribution         |
| `ask1`               | Highest previous contribution (for suggestion)                      |
| `ask2`               | 1.25 x highest previous contribution (for suggestion)               |
| `ask3`               | 1.50 x highest previous contribution (for suggestion)               |
| `amount`             | Dollars given                                                       |
| `gave`               | Gave anything                                                       |
| `amountchange`       | Change in amount given                                              |
| `hpa`                | Highest previous contribution                                       |
| `ltmedmra`           | Small prior donor: last gift was less than median \$35              |
| `freq`               | Number of prior donations                                           |
| `years`              | Number of years since initial donation                              |
| `year5`              | At least 5 years since initial donation                             |
| `mrm2`               | Number of months since last donation                                |
| `dormant`            | Already donated in 2005                                             |
| `female`             | Female                                                              |
| `couple`             | Couple                                                              |
| `state50one`         | State tag: 1 for one observation of each of 50 states; 0 otherwise  |
| `nonlit`             | Nonlitigation                                                       |
| `cases`              | Court cases from state in 2004-5 in which organization was involved |
| `statecnt`           | Percent of sample from state                                        |
| `stateresponse`      | Proportion of sample from the state who gave                        |
| `stateresponset`     | Proportion of treated sample from the state who gave                |
| `stateresponsec`     | Proportion of control sample from the state who gave                |
| `stateresponsetminc` | stateresponset - stateresponsec                                     |
| `perbush`            | State vote share for Bush                                           |
| `close25`            | State vote share for Bush between 47.5% and 52.5%                   |
| `red0`               | Red state                                                           |
| `blue0`              | Blue state                                                          |
| `redcty`             | Red county                                                          |
| `bluecty`            | Blue county                                                         |
| `pwhite`             | Proportion white within zip code                                    |
| `pblack`             | Proportion black within zip code                                    |
| `page18_39`          | Proportion age 18-39 within zip code                                |
| `ave_hh_sz`          | Average household size within zip code                              |
| `median_hhincome`    | Median household income within zip code                             |
| `powner`             | Proportion house owner within zip code                              |
| `psch_atlstba`       | Proportion who finished college within zip code                     |
| `pop_propurban`      | Proportion of population urban within zip code                      |

::::
 

### Balance Test 

As an ad hoc test of the randomization mechanism, I provide a series of tests that compare aspects of the treatment and control groups to assess whether they are statistically significantly different from one another.


In [ ]:
import statsmodels.api as sm
from scipy import stats


df = pd.read_stata("karlan_list_2007.dta")

# T-test and regression for mrm2
treat = df[df['treatment'] == 1]['mrm2']
control = df[df['treatment'] == 0]['mrm2']
t_stat1, p_val1 = stats.ttest_ind(treat, control, nan_policy='omit')

df['intercept'] = 1
model1 = sm.OLS(df['mrm2'], df[['intercept', 'treatment']], missing='drop').fit()

# T-test and regression for amount
treat_amt = df[df['treatment'] == 1]['amount']
control_amt = df[df['treatment'] == 0]['amount']
t_stat2, p_val2 = stats.ttest_ind(treat_amt, control_amt, nan_policy='omit')

model2 = sm.OLS(df['amount'], df[['intercept', 'treatment']], missing='drop').fit()


print("=== mrm2 (Months Since Last Donation) ===")
print(f"T-test: T-stat = {t_stat1:.4f}, P = {p_val1:.4f}")
print(model1.summary())

print("\n=== amount (Previous Donation Amount) ===")
print(f"T-test: T-stat = {t_stat2:.4f}, P = {p_val2:.4f}")
print(model2.summary())

These results show that randomization appears to have worked. There are no statistically significant differences between the treatment and control groups on these baseline characteristics. This is important because it supports the internal validity of the experiment — we can reasonably believe that any differences in donation outcomes later on were caused by the treatment and not by pre-existing differences.

This is exactly why Table 1 is included in Karlan & List (2007) — to show that the treatment assignment was random and the groups were comparable at baseline.

## Experimental Results

### Charitable Contribution Made

First, I analyze whether matched donations lead to an increased response rate of making a donation. 


In [ ]:
import matplotlib.pyplot as plt


df['donated'] = (df['amount'] > 0).astype(int)


donation_rates = df.groupby('treatment')['donated'].mean()

# Bar plot
plt.bar(['Control', 'Treatment'], donation_rates)
plt.ylabel('Proportion Donated')
plt.title('Donation Rate by Treatment Group')
plt.ylim(0, 0.15)
plt.grid(axis='y')
plt.show()

In [ ]:
treat_d = df[df['treatment'] == 1]['donated']
control_d = df[df['treatment'] == 0]['donated']
t_stat, p_val = stats.ttest_ind(treat_d, control_d)
print(f"T-test: t = {t_stat:.4f}, p = {p_val:.4f}")


model = sm.OLS(df['donated'], df[['intercept', 'treatment']], missing='drop').fit()
print(model.summary())

In [ ]:
probit = sm.Probit(df['donated'], df[['intercept', 'treatment']], missing='drop').fit()
print(probit.summary())

To see if matching donations increased the chance someone donated at all, I compared donation rates between the treatment and control groups.

The bar plot shows a clear difference: people in the treatment group donated at a higher rate than those in the control group.

Using both a t-test and a linear regression, I confirmed that this difference is statistically significant. This means the treatment group, who received a matching offer, was more likely to make any donation.

I also ran a probit regression, which models the probability of making a donation. The results again showed that being assigned to the treatment group had a positive effect on whether someone donated.

What we learn
This tells us something important about human behavior: people are more likely to give when they know their donation will be matched. Even though the match doesn't change their actual out-of-pocket cost, it creates a stronger sense of impact. That seems to motivate giving.

In the context of charitable giving, this suggests that match offers are an effective tool to increase participation, not just donation amounts. This aligns with the results shown in Table 2a Panel A and Table 3 of the paper.
_NOTE: Linear regression results appear replicate Table 3 column 1 in the paper. Probit results do not, despite Table 3 indicating its results come from probit regressions..._


### Differences between Match Rates

Next, I assess the effectiveness of different sizes of matched donations on the response rate.

In [ ]:
df['gave'] = (df['amount'] > 0).astype(int)


df['ratio1'] = (df['ratio'] == '1:1').astype(int)
df['ratio2'] = (df['ratio'] == '2:1').astype(int)
df['ratio3'] = (df['ratio'] == '3:1').astype(int)

In [ ]:
from scipy import stats

# Response rates by match ratio
rate_1_1 = df[df['ratio'] == '1:1']['gave']
rate_2_1 = df[df['ratio'] == '2:1']['gave']
rate_3_1 = df[df['ratio'] == '3:1']['gave']

# T-tests between groups
t12, p12 = stats.ttest_ind(rate_1_1, rate_2_1, nan_policy='omit')
t23, p23 = stats.ttest_ind(rate_2_1, rate_3_1, nan_policy='omit')

print(f"1:1 vs 2:1 — T = {t12:.4f}, P = {p12:.4f}")
print(f"2:1 vs 3:1 — T = {t23:.4f}, P = {p23:.4f}")

In [ ]:
# Add intercept
df['intercept'] = 1

# Regression on dummy variables (1:1 is omitted)
reg = sm.OLS(df['gave'], df[['intercept', 'ratio2', 'ratio3']], missing='drop').fit()
print(reg.summary())

In [ ]:
mean_1_1 = rate_1_1.mean()
mean_2_1 = rate_2_1.mean()
mean_3_1 = rate_3_1.mean()

print(f"Response rate (1:1): {mean_1_1:.4f}")
print(f"Response rate (2:1): {mean_2_1:.4f}")
print(f"Response rate (3:1): {mean_3_1:.4f}")
print(f"2:1 - 1:1 difference: {mean_2_1 - mean_1_1:.4f}")
print(f"3:1 - 2:1 difference: {mean_3_1 - mean_2_1:.4f}")

### Size of Charitable Contribution

In this subsection, I analyze the effect of the size of matched donation on the size of the charitable contribution.


In [ ]:
# Regression on all observations
model_all = sm.OLS(df['amount'], df[['intercept', 'treatment']], missing='drop').fit()
print(model_all.summary())

# T-test as a double check
t_all, p_all = stats.ttest_ind(
    df[df['treatment'] == 1]['amount'],
    df[df['treatment'] == 0]['amount'],
    nan_policy='omit'
)
print(f"T-test (all): t = {t_all:.4f}, p = {p_all:.4f}")

In [ ]:
# Subset to only donors
donors = df[df['amount'] > 0]

# Regression on donors only
model_donors = sm.OLS(donors['amount'], donors[['intercept', 'treatment']], missing='drop').fit()
print(model_donors.summary())

# Optional: t-test for donors only
t_don, p_don = stats.ttest_ind(
    donors[donors['treatment'] == 1]['amount'],
    donors[donors['treatment'] == 0]['amount'],
    nan_policy='omit'
)
print(f"T-test (donors only): t = {t_don:.4f}, p = {p_don:.4f}")

In [ ]:
import matplotlib.pyplot as plt

# Histograms by group
fig, axs = plt.subplots(1, 2, figsize=(12, 5), sharey=True)

# Control group
control_don = donors[donors['treatment'] == 0]['amount']
axs[0].hist(control_don, bins=30, color='skyblue')
axs[0].axvline(control_don.mean(), color='red', linestyle='dashed', linewidth=2)
axs[0].set_title('Control Group Donations')
axs[0].set_xlabel('Donation Amount')
axs[0].set_ylabel('Frequency')

# Treatment group
treat_don = donors[donors['treatment'] == 1]['amount']
axs[1].hist(treat_don, bins=30, color='lightgreen')
axs[1].axvline(treat_don.mean(), color='red', linestyle='dashed', linewidth=2)
axs[1].set_title('Treatment Group Donations')
axs[1].set_xlabel('Donation Amount')

plt.tight_layout()
plt.show()

## Simulation Experiment

As a reminder of how the t-statistic "works," in this section I use simulation to demonstrate the Law of Large Numbers and the Central Limit Theorem.

Suppose the true distribution of respondents who do not get a charitable donation match is Bernoulli with probability p=0.018 that a donation is made. 

Further suppose that the true distribution of respondents who do get a charitable donation match of any size is Bernoulli with probability p=0.022 that a donation is made.

### Law of Large Numbers


In [ ]:
import numpy as np


np.random.seed(42)

# Simulate 10,000 Bernoulli draws for control (p=0.018) and treatment (p=0.022)
control_draws = np.random.binomial(1, 0.018, 10000)
treatment_draws = np.random.binomial(1, 0.022, 10000)

# Calculate difference at each pair of draws
differences = treatment_draws - control_draws

# Compute cumulative average of the differences
cumulative_avg = np.cumsum(differences) / np.arange(1, len(differences) + 1)

# Plot the cumulative average
plt.figure(figsize=(10, 5))
plt.plot(cumulative_avg, color='navy')
plt.axhline(0.004, color='red', linestyle='--', label='True Avg Difference (0.004)')
plt.title("Cumulative Average Difference (LLN in Action)")
plt.xlabel("Number of Observations")
plt.ylabel("Cumulative Average (Treatment - Control)")
plt.legend()
plt.grid(True)
plt.show()

To show how the Law of Large Numbers (LLN) works, I simulated 10,000 random draws from each group’s true donation distribution:

Control group had a 1.8% chance of donating (p = 0.018)

Treatment group had a 2.2% chance (p = 0.022)

At each draw, I calculated the difference between treatment and control and then plotted the cumulative average of those differences.

What we learn:
Early in the plot, the average difference jumps around — this is due to random noise when sample sizes are small.

But as more data is added, the average settles down near the true expected treatment effect: 0.004.

This is the Law of Large Numbers in action: with more observations, our sample average converges to the true population average.

This helps explain why large sample sizes make our experimental estimates more reliable and less sensitive to random variation.

### Central Limit Theorem


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)


sample_sizes = [50, 200, 500, 1000]
simulations = 1000


p_control = 0.018
p_treat = 0.022

# Prepare plots
fig, axs = plt.subplots(2, 2, figsize=(12, 8))
axs = axs.flatten()

for i, n in enumerate(sample_sizes):
    diffs = []
    for _ in range(simulations):
        c = np.random.binomial(1, p_control, n)
        t = np.random.binomial(1, p_treat, n)
        diffs.append(t.mean() - c.mean())

    axs[i].hist(diffs, bins=30, color='skyblue', edgecolor='black')
    axs[i].axvline(0.004, color='red', linestyle='--', label='True Avg Diff (0.004)')
    axs[i].set_title(f"Sample Size = {n}")
    axs[i].set_xlabel("Avg Difference (Treatment - Control)")
    axs[i].set_ylabel("Frequency")
    axs[i].legend()

plt.tight_layout()
plt.show()

To visualize the Central Limit Theorem (CLT), I created four histograms showing the sampling distribution of the average difference in donation rates between the treatment and control groups. For each sample size — 50, 200, 500, and 1000 — I simulated 1000 experiments and plotted the results.

What we learn:
With small samples (like 50), the distribution is wide and irregular — more influenced by random chance.

As the sample size increases, the distributions become tighter and more bell-shaped.

By the time we reach 1000 draws, the sampling distribution is narrow and symmetric, closely resembling a normal distribution centered near the true difference (0.004).

This shows the Central Limit Theorem in action: as the number of observations increases, the average of random variables (in this case, donation differences) becomes more predictable and normally distributed — even if the original data is binary.

This helps us understand why t-tests and confidence intervals work well in large-sample experiments — because the underlying averages behave in a stable, normal way.
